
Create a runner notebook using papermill to run the template notebook across all tokens in the database and the last 2 complete weeks in the database.  Summarize the following in a single DataFrame in the runner notebook
    - avg hourly return by run
    - avg volatility by run
    - maximum drawdown over the entire period of each run
    - range of last close price - first close price


In [1]:
import papermill as pm
import scrapbook as sb
import pandas as pd
import time
from datetime import datetime, timedelta
import os

/Users/sundaylai/opt/anaconda3/envs/lecture_4/lib/python3.9/site-packages/papermill/iorw.py:50: FutureWarning: pyarrow.HadoopFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
  from pyarrow import HadoopFileSystem


In [2]:
def last_dow(d: datetime, dow: int) -> datetime:
    '''
    Gets the date of the last day of week, using datetime convention of Monday is 0 and Sunday is 6
    '''
    dow_d = d.weekday()
    days_delta = (7 - dow) + dow_d if dow_d < dow else dow_d - dow
    
    return d - timedelta(days=days_delta)

In [3]:
prices = pd.read_sql(f'''
SELECT 
    * 
FROM ohlc
''', 'sqlite:///data/data.db')
prices['ts'] = pd.to_datetime(prices['ts'])

prices.set_index('ts', inplace=True)

In [4]:
prices.token.unique()

array(['BTC', 'ETH', 'USDT', 'SOL', 'ADA', 'DOT', 'AVAX', 'ATOM', 'CRV',
       'AAVE', 'COMP'], dtype=object)

In [5]:
run_dates = [(last_dow(datetime(2021, 11, 5), 4) + timedelta(days=7 * x)) for x in range(1)]
tokens = ['BTC', 'ETH', 'USDT', 'SOL', 'ADA', 'DOT', 'AVAX', 'ATOM', 'CRV',
       'AAVE', 'COMP']
connection_string = 'sqlite:///data/data.db'
events_db = 'crypto_events_database'

In [6]:
base_output_folder = os.path.expanduser('outputs')

if not os.path.exists(base_output_folder):
        os.makedirs(base_output_folder)

In [7]:
run_dates

[datetime.datetime(2021, 11, 5, 0, 0)]

In [8]:
for run_date in run_dates:
    for token in tokens:
        print(f'running for {token} for date {run_date} to {run_date + timedelta(days=7)}', end='\r')
        res = pm.execute_notebook(
            'hw4_sunday_lai_template.ipynb',
            f'{base_output_folder}/market_analysis_{token}_{run_date}.ipynb',
            parameters = {
                'from_date': f'{run_date}',
                'to_date': f'{run_date + timedelta(days=7)}',
                'token': token,
                'connection_string': connection_string,
                'events_db': events_db
            }
        )

Executing:   0%|          | 0/18 [00:00<?, ?cell/s]

Executing:   0%|          | 0/18 [00:00<?, ?cell/s]

Executing:   0%|          | 0/18 [00:00<?, ?cell/s]

Executing:   0%|          | 0/18 [00:00<?, ?cell/s]

Executing:   0%|          | 0/18 [00:00<?, ?cell/s]

...

Executing:   0%|          | 0/18 [00:00<?, ?cell/s]

Executing:   0%|          | 0/18 [00:00<?, ?cell/s]

Executing:   0%|          | 0/18 [00:00<?, ?cell/s]

Executing:   0%|          | 0/18 [00:00<?, ?cell/s]

Executing:   0%|          | 0/18 [00:00<?, ?cell/s]

In [9]:
nbs = sb.read_notebooks('outputs')

In [10]:
pd.DataFrame([
    pd.Series({
        'token': scrap['token'].data,
        'from_date': scrap['from_date'].data,
        'to_date': scrap['to_date'].data,
        'average hourly return': scrap['prices'].data['hourly return'].mean(),
        'average volatility': scrap['prices'].data.volatility.mean(),
        'maximum drawdown':scrap['prices'].data['maximum drawdown'][-1],
        'range': scrap['prices'].data.close[-1] - scrap['prices'].data.close[0]}) 
    for book, scrap in nbs.notebook_scraps.items()
])

,token,from_date,to_date,average hourly return,average volatility,maximum drawdown,range
0,AAVE,2021-11-05 00:00:00,2021-11-12 00:00:00,-4.600160e-04,0.151366,13.7630,-19.5870
1,ADA,2021-11-05 00:00:00,2021-11-12 00:00:00,3.472630e-04,0.160093,0.0869,0.1210
2,ATOM,2021-11-05 00:00:00,2021-11-12 00:00:00,-6.109935e-04,0.187634,2.8010,-4.1560
3,AVAX,2021-11-05 00:00:00,2021-11-12 00:00:00,7.604500e-04,0.262145,5.0000,10.1100
4,BTC,2021-11-05 00:00:00,2021-11-12 00:00:00,3.784540e-04,0.095865,1866.0700,3759.0200
5,COMP,2021-11-05 00:00:00,2021-11-12 00:00:00,-5.639838e-04,0.157245,18.7700,-29.0500
6,CRV,2021-11-05 00:00:00,2021-11-12 00:00:00,9.030432e-06,0.237496,0.3344,0.0835
7,DOT,2021-11-05 00:00:00,2021-11-12 00:00:00,-6.460512e-04,0.171455,2.2860,-5.8260
8,ETH,2021-11-05 00:00:00,2021-11-12 00:00:00,2.924078e-04,0.099854,105.7700,215.6500
9,SOL,2021-11-05 00:00:00,2021-11-12 00:00:00,-2.991968e-04,0.172281,8.8490,-10.9980
